In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

MessageError: ignored

In [4]:
! pip install -q pytorch-lightning

     |████████████████████████████████| 798 kB 28.1 MB/s 
     |████████████████████████████████| 125 kB 56.7 MB/s 
     |████████████████████████████████| 512 kB 47.4 MB/s 
     |████████████████████████████████| 87 kB 7.1 MB/s 


In [5]:
! pip install -q bs4
! pip install -q transformers

     |████████████████████████████████| 5.5 MB 26.4 MB/s 
     |████████████████████████████████| 182 kB 65.4 MB/s 
     |████████████████████████████████| 7.6 MB 47.4 MB/s 


In [1]:
# Import all libraries
import pandas as pd
import numpy as np
import re


In [2]:
# Huggingface transformers
import transformers
from transformers import BertModel,BertTokenizer,AdamW, get_linear_schedule_with_warmup

ModuleNotFoundError: ignored

In [8]:

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

In [9]:

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [11]:

#handling html data
from bs4 import BeautifulSoup


In [12]:

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [13]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
from google.colab import files
 
 
uploaded = files.upload()

Saving StackoverFlow_Final.csv to StackoverFlow_Final.csv


In [16]:
import io
df = pd.read_csv(io.BytesIO(uploaded['StackoverFlow_Final.csv']))
print(df)

       Unnamed: 0        Id  Score  \
0               0        80     26   
1               1        90    144   
2               2       120     21   
3               3       180     53   
4               4       260     49   
...           ...       ...    ...   
71232     1262915  40111730      7   
71233     1263065  40115300      7   
71234     1263399  40123880      8   
71235     1263454  40125310      6   
71236     1264205  40143070      6   

                                                   Title  \
0      SQLStatement.execute() - multiple queries in o...   
1      Good branching and merging tutorials for Torto...   
2                                      ASP.NET Site Maps   
3                     Function for creating color wheels   
4      Adding scripting functionality to .NET applica...   
...                                                  ...   
71232           How to use a dict to subset a DataFrame?   
71233  Is there a way to use itertools in python to c...   
712

In [17]:
df.head()

,Unnamed: 0,Id,Score,Title,Body,Tags,title_clean,body_clean
0,0,80,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in a...,"['flex', 'actionscript-3']",sqlstatementexecut multipl queri one statement,i'v written databas gener script want execut a...
1,1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,"['svn', 'tortoisesvn', 'branch']",good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...
2,2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"['sql', 'asp.net']",aspnet site map,anyon got experi creat sql-base asp.net site-m...
3,3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,"['algorithm', 'language-agnostic', 'colors']",function creat color wheel,someth i'v pseudo-solv mani time never quit fo...
4,4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"['c#', '.net', 'scripting', 'compiler-construc...",ad script function net applic,"littl game written c#. use databas back-end. ,..."


In [18]:
# count of unique tags
len(df['Tags'].unique())

32156

In [19]:
df['Tags'].value_counts()[0:10]

['android']                 1350
['java']                     838
['python']                   829
['c#']                       684
['javascript']               622
['c++']                      502
['php']                      479
['git']                      432
['r']                        386
['javascript', 'jquery']     364
Name: Tags, dtype: int64

In [20]:
# remove "-" from the tags
df['Tags']= df['Tags'].apply(lambda x:re.sub("-"," ",x))

In [21]:
# Consider only Top 10 tags - want to keep a smaller dataset
top_tags = df['Tags'].value_counts().keys()[0:10]
top_tags

Index(['['android']', '['java']', '['python']', '['c#']', '['javascript']',
       '['c++']', '['php']', '['git']', '['r']', '['javascript', 'jquery']'],
      dtype='object')

In [22]:
# First group tags Id wise
df1 = df.groupby('Id').apply(lambda x:x['Tags'].values).reset_index(name='tags')
df1.head()

,Id,tags
0,80,"[['flex', 'actionscript 3']]"
1,90,"[['svn', 'tortoisesvn', 'branch']]"
2,120,"[['sql', 'asp.net']]"
3,180,"[['algorithm', 'language agnostic', 'colors']]"
4,260,"[['c#', '.net', 'scripting', 'compiler constru..."


In [23]:
# merging title and body
df['Text'] =df['body_clean']+df['title_clean']

In [24]:
# merge tags and questions datasets
df = pd.merge(df,df1,how='inner',on='Id')

In [25]:
df.head()

,Unnamed: 0,Id,Score,Title,Body,Tags,title_clean,body_clean,Text,tags
0,0,80,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in a...,"['flex', 'actionscript 3']",sqlstatementexecut multipl queri one statement,i'v written databas gener script want execut a...,i'v written databas gener script want execut a...,"[['flex', 'actionscript 3']]"
1,1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,"['svn', 'tortoisesvn', 'branch']",good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...,realli good tutori explain apach subversion? b...,"[['svn', 'tortoisesvn', 'branch']]"
2,2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"['sql', 'asp.net']",aspnet site map,anyon got experi creat sql-base asp.net site-m...,anyon got experi creat sql-base asp.net site-m...,"[['sql', 'asp.net']]"
3,3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,"['algorithm', 'language agnostic', 'colors']",function creat color wheel,someth i'v pseudo-solv mani time never quit fo...,someth i'v pseudo-solv mani time never quit fo...,"[['algorithm', 'language agnostic', 'colors']]"
4,4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"['c#', '.net', 'scripting', 'compiler construc...",ad script function net applic,"littl game written c#. use databas back-end. ,...","littl game written c#. use databas back-end. ,...","[['c#', '.net', 'scripting', 'compiler constru..."


In [26]:
# Filter out records ( values in clean_body and tags) that have atleast one of the top tags

x=[] # To store the filtered clean_body values
y=[] # to store the corresponding tags
# Convert to list data type
lst_top_tags = list(top_tags)

for i in range(len(df['tags'])):
    temp=[]
    for tag in df['tags'][i]:
        if tag in lst_top_tags:
            temp.append(tag)

    if(len(temp)>0):
        x.append(df['body_clean'][i])
        y.append(temp)

In [27]:
# number of records that will be used for training and testing
len(x)

6486

In [28]:
# Encode the tags(labels) in a binary format in order to be used for training

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
 
yt = mlb.fit_transform(y)
yt.shape

(6486, 10)

In [29]:
# Getting a sense of how the tags data looks like
print(yt[0])
print(mlb.inverse_transform(yt[0].reshape(1,-1)))
print(mlb.classes_)

[0 1 0 0 0 0 0 0 0 0]
[("['c#']",)]
["['android']" "['c#']" "['c++']" "['git']" "['java']"
 "['javascript', 'jquery']" "['javascript']" "['php']" "['python']"
 "['r']"]


Split the dataset into training ,validation and test set.


In [30]:
from sklearn.model_selection import train_test_split
# First Split for Train and Test
x_train,x_test,y_train,y_test = train_test_split(x, yt, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x_tr,x_val,y_tr,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

In [31]:
len(x_tr) ,len(x_val), len(x_test)

(4669, 1168, 649)

Preparing the Dataset and DataModule
First create QTagDataset class based on the Dataset class,that readies the text in a format needed for the BERT Model

In [32]:
class QTagDataset (Dataset):
    def __init__(self,quest,tags, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = quest
        self.labels = tags
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        #token_type_ids = inputs["token_type_ids"]
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.labels[item_idx], dtype=torch.float)
            
        }

In [33]:
#Since we are using Pytorch Lightning for Model training - we will setup the QTagDataModule class that is derived from the LightningDataModule
class QTagDataModule (pl.LightningDataModule):
    
    def __init__(self,x_tr,y_tr,x_val,y_val,x_test,y_test,tokenizer,batch_size=16,max_token_len=200):
        super().__init__()
        self.tr_text = x_tr
        self.tr_label = y_tr
        self.val_text = x_val
        self.val_label = y_val
        self.test_text = x_test
        self.test_label = y_test
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self):
        self.train_dataset = QTagDataset(quest=self.tr_text, tags=self.tr_label, tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.val_dataset  = QTagDataset(quest=self.val_text,tags=self.val_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.test_dataset  = QTagDataset(quest=self.test_text,tags=self.test_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        
        
    def train_dataloader(self):
        return DataLoader (self.train_dataset,batch_size = self.batch_size,shuffle = True , num_workers=2)

    def val_dataloader(self):
        return DataLoader (self.val_dataset,batch_size= 16)

    def test_dataloader(self):
        return DataLoader (self.test_dataset,batch_size= 16)

In [34]:
# Initialize the Bert tokenizer
BERT_MODEL_NAME = "bert-base-cased" # we will use the BERT base model(the smaller one)
Bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [35]:
# Initialize the parameters that will be use for training
N_EPOCHS = 12
BATCH_SIZE = 32
MAX_LEN = 300
LR = 2e-05

In [36]:
# Instantiate and set up the data_module
QTdata_module = QTagDataModule(x_tr,y_tr,x_val,y_val,x_test,y_test,Bert_tokenizer,BATCH_SIZE,MAX_LEN)
QTdata_module.setup()

Train the Model
Setup the Multi-label Classifier Model - dervived from LightningModule , similar to nn.module of PyTorch

In [37]:
class QTagClassifier(pl.LightningModule):
    # Set up the classifier
    def __init__(self, n_classes=10, steps_per_epoch=None, n_epochs=3, lr=2e-5 ):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size,n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        output = self.classifier(output.pooler_output)
                
        return output
    
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss

    def test_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('test_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr)
        warmup_steps = self.steps_per_epoch//3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,total_steps)

        return [optimizer], [scheduler]

In [38]:
# Instantiate the classifier model
steps_per_epoch = len(x_tr)//BATCH_SIZE
model = QTagClassifier(n_classes=10, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
# Instantiate the classifier model
steps_per_epoch = len(x_tr)//BATCH_SIZE
model = QTagClassifier(n_classes=10, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',# monitored quantity
    filename='QTag-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3, #  save the top 3 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [48]:
# Instantiate the Model Trainer

trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = 1, callbacks=[checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


MisconfigurationException: ignored